<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S105ocrCorrectionV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# correction of ocr-generated text 
- using correction dictionaries 
- using rules


In [1]:
!wget https://heibox.uni-heidelberg.de/f/d601ceb0af5a4671a8e7/?dl=1
!mv index.html?dl=1 Parfum_Arm_ABBY.txt

!wget https://heibox.uni-heidelberg.de/f/743a1a57a37c42d8b585/?dl=1
!mv index.html?dl=1 Parfum_Armenian_uncorrected.txt



--2023-03-20 11:48:02--  https://heibox.uni-heidelberg.de/f/d601ceb0af5a4671a8e7/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/b99b41c7-b368-4662-b8cd-72e45fe53a42/Parfum_Arm_ABBY.txt [following]
--2023-03-20 11:48:03--  https://heibox.uni-heidelberg.de/seafhttp/files/b99b41c7-b368-4662-b8cd-72e45fe53a42/Parfum_Arm_ABBY.txt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 867013 (847K) [text/plain]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 846.69K   786KB/s    in 1.1s    

2023-03-20 11:48:04 (786 KB/s) - ‘index.html?dl=1’ saved [867013/867013]

--2023-03-20 11:48:04--  https://heibox.uni-heidelberg.de/f/743a1a57a37c42d8b585/?dl=1
Resolving hei

In [2]:
# dealing with Armenian OCR output with line breaks (is it correct?)

FName = 'Parfum_Armenian_uncorrected.txt'
FNameOut = 'Parfum_Armenian.txt'

FIn = open(FName, 'r')
FOut = open(FNameOut, 'w')

for SLine in FIn:
    SLine = SLine.strip()
    if SLine == '': 
        FOut.write('\n\n')
        continue
    if SLine[-1] == '-':
        SLine2write = SLine[:-1]
        FOut.write(SLine2write)
        continue

    FOut.write(SLine + ' ')
FOut.flush()



In [3]:
!wc Parfum_Arm_ABBY.txt
!wc Parfum_Armenian_uncorrected.txt
!wc Parfum_Armenian.txt

  1489  68852 867013 Parfum_Arm_ABBY.txt
 13592  72207 854251 Parfum_Armenian_uncorrected.txt
  6126  69006 849250 Parfum_Armenian.txt


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/c977e87cf2b244e6801b/?dl=1
!mv index.html?dl=1 KorpusARM.tgz


In [ ]:
!tar xvzf KorpusARM.tgz
!mkdir KorpusARM1
!mkdir KorpusARM1/stage01
# concatenating files
!cat korpusARM/hyFiktion/* >KorpusARM1/stage01/hyFiktion.txt
!cat korpusARM/hyNatur/* >KorpusARM1/stage01/hyNatur.txt
!cat korpusARM/hyRecht/* >KorpusARM1/stage01/hyRecht.txt
!mkdir KorpusARM1/stage02

# function for Armenian line breaks:

def correctLineBreaksHY(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')
    countHyphens = 0
    for SLine in FIn:
        SLine = SLine.strip()
        if SLine == '': 
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            countHyphens +=1
            continue
        FOut.write(SLine + ' ')
    FOut.flush()
    print(str(countHyphens) + ' hyphens corrected')
    return

correctLineBreaksHY('KorpusARM1/stage01/hyFiktion.txt', 'KorpusARM1/stage02/hyFiktion.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyNatur.txt', 'KorpusARM1/stage02/hyNatur.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyRecht.txt', 'KorpusARM1/stage02/hyRecht.txt')


In [6]:

!wc KorpusARM1/stage02/hyFiktion.txt
!wc KorpusARM1/stage02/hyNatur.txt
!wc KorpusARM1/stage02/hyRecht.txt

   6208   92131 1081755 KorpusARM1/stage02/hyFiktion.txt
  3642  67142 870081 KorpusARM1/stage02/hyNatur.txt
   8940   86621 1288655 KorpusARM1/stage02/hyRecht.txt


In [ ]:
# Dowloading the file with corrections
!wget https://heibox.uni-heidelberg.de/f/14706c04a4024b2f937d/?dl=1
!mv index.html?dl=1 Pilot-Corrections-all.tsv
!wc Pilot-Corrections-all.tsv

In [17]:
def readCorrections(colNumberOri, colNumberCorrect, SFIn, SFOut = None):
    LTWrongCorrect = []
    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.strip()
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\n')    
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect

def readCorrectionsFrq(colNumberOri, colNumberCorrect, colNumberFrq, SFIn, SFOut = None):
    LTWrongCorrect = []
    '''
    if type(LTWrongCorrect) == list:
        pass
    '''

    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.rstrip('\n')
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            SFrq = LLine[colNumberFrq]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]', f'{SFrq}')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect, SFrq in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\t{SFrq}\n')    
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect


In [18]:
LTWrongCorrectWordF, DWrongCorrectWordF = readCorrectionsFrq(1, 4, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-WordForm.tsv')
# LTWrongCorrectLemmaF, DWrongCorrectLemmaF = readCorrectionsFrq(3, 6, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-Lemma.tsv')
print(LTWrongCorrectWordF)
# print(LTWrongCorrectLemmaF)
# ինձ|ինչ
# առջև|առջևից
# ինչ|ինձ
# ինչ|ես
# գիտենալ|իմանալ


ինճ	ինձ	ինչ
առջնից	առջև	առջևից
ինճ	ինչ	ինձ
172
[('[մերճակա]', '[մերձակա]', '4'), ('[առջն]', '[առջև]', '4'), ('[թեթնություն]', '[թեթևություն]', '4'), ('[Եթենա]', '[եթե նա]', '4'), ('[ննա]', '[նա]', '4'), ('[ճեռքերն]', '[ձեռքից]', '4'), ('[ճայն]', '[ձայն]', '4'), ('[ճեռքով]', '[ձեռքով]', '4'), ('[այլնս]', '[այլևս]', '4'), ('[ետնի]', '[ետևի]', '4'), ('[կեղնները]', '[կեղևները]', '4'), ('[ճկան]', '[ձկան]', '4'), ('[ննա]', '[նա]', '4'), ('[բանաձնի]', '[բանաձևի]', '4'), ('[ճեր]', '[ձեր]', '4'), ('[առնտրական]', '[առևտրական]', '4'), ('[արնելյան]', '[արևելյան]', '4'), ('[կուղնորվի]', '[կուղևորվի]', '4'), ('[նս]', '[ևս]', '4'), ('[հետնեց]', '[հետևել]', '4'), ('[նուխիսկ]', '[նույնիսկ]', '4'), ('[երնույթ]', '[երևույթ]', '4'), ('[քրտնքով]', '[քրտինքով]', '4'), ('[արվարճանում]', '[արվարձանում]', '4'), ('[անճամբ]', '[անձամբ]', '4'), ('[ճայնը]', '[ձայնը]', '4'), ('[ճգվում]', '[ձգվում]', '4'), ('[ճիու]', '[ձիու]', '4'), ('[դարճնում]', '[դարձնում]', '4'), ('[ուղնորվում]', '[ուղևորվում ]', '4'), ('[իջնանա

In [19]:
print(len(DWrongCorrectWordF))
# print(len(DWrongCorrectLemmaF))


172


In [23]:
for key, value in sorted(DWrongCorrectWordF.items()):
    print(f'{key}\t{value}')

Այլնս	այլևս
Դետեսնում	դե տեսնում 
Դհյոյում	Դյո
Եթենա	եթե նա
Թերնս	թերևս
Թռենել	թռնել
Հետնաբար	հետևաբար
Հնայած	Չնայած
Միգուցենա	միգուցե նա
Նախնառաջ	նախևառաջ
Շավանաբար	հավանաբար
Որովհետն	որովհետև
Ջգազմունքներից	Զգացմունք
այլես	այլևս
այլնայլ	այլևայլ
այլնս	այլևս
այխքան	այդքան
այնուհետն	այնուհետև
անձր	անձ, անձրև
անձրնը	անձրևը
անճամբ	անձամբ
անճնական	անձնական
առանճին	առանձին
առնտուրը	առևտուրը
առնտրական	առևտրական
առջն	առջև
առջնից	առջևից
առջնում	առջևում
աստվածավախու	աստվածավախություն
արճագանք	արձագանք
արճակել	արձակել
արճակող	արձակող
արճակում	արձակում
արճակվող	արձակվել
արնելյան	արևելյան
արնելք	արևելք
արնի	արևի
արնմուտք	արևմուտք
արնմուտքից	արևմուտքից
արնմտյան	արևմտյան
արվարճանի	արվարձանի
արվարճանում	արվարձանում
բանաձն	բանաձևն
բանաձնը	բանաձևը
բանաձնի	բանաձևի
բանաճն	բանաձևն
բանաճներ	բանաձևեր
բանաճներով	բանաձներով
բանաճնը	բանաձևը
բանաճնի	բանաձևի
բարճունքներում	բարձունքներում
բարճր	բարձր
բարճրաց	բարձրացնել
բարճրացավ	բարձրանալ
բարճրացնել	բարձրացնել
բացարճակ	բացարձակ
բացարճակապես	բացարձակապես
բնեռը	բևե

In [ ]:
'''
for i in range(5):
    LKeys = list(DWrongCorrectLemmaF.keys())
    SKey = LKeys[i]
    SVal = DWrongCorrectWordF[SKey]
    print(f'{SKey} {SVal}\n')
'''
 

In [11]:
import re, os, sys

def tokenizeTextHY(SFIn):
    LLParagraphs = []
    with open(SFIn, 'r') as FIn:
        for SLine in FIn:
            SLine = SLine.strip()
            LLine = re.split('([ ,\.\:։;\'\"\(\)\-\–\!\?\{\}\t\«\»]+)', SLine)
            if LLine: LLParagraphs.append(LLine)
    return LLParagraphs

def printLLParagraphs(SFIn, SFOut, DCorrections1 = None, DCorrections2 = None):
    LLParagraphs = tokenizeTextHY(SFIn)
    LCorrected = []
    FOut = open(SFOut, 'w')
    for LPara in LLParagraphs:
        LParaNew = []
        FOut.write('<p>\n')
        
        for el in LPara:
            # if el == ' ': continue
            if DCorrections1:
                if el in DCorrections1:
                    elCorrect = DCorrections1[el]
                    LCorrected.append((el, elCorrect, LPara))
                else:
                    elCorrect = el
            if DCorrections2:
                if el in DCorrections2:
                    elCorrect = DCorrections2[el]
                    LCorrected.append((el, elCorrect, LPara))
                else:
                    elCorrect = el                    
            LParaNew.append(elCorrect)

        SParaNew = ' '.join(LParaNew)
        FOut.write(f'{SParaNew}\n')

        FOut.write('</p>\n')
    FOut.flush()
    return LCorrected

In [13]:
LCorrected1 = printLLParagraphs('/content/Parfum_Arm_ABBY.txt', '/content/Parfum_Arm_ABBY-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
if LCorrected1:
    FOut1 = open('/content/Parfum_Arm_ABBY-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )

LCorrected2 = printLLParagraphs('/content/Parfum_Armenian.txt', '/content/Parfum_Armenian-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected2))
if LCorrected2:
    FOut2 = open('/content/Parfum_Armenian-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut2.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )


6
918


In [14]:
LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyFiktion.txt', '/content/KorpusARM1/stage02/hyFiktion-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
print(len(LCorrected1))
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyFiktion-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )

3660


In [15]:
LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyNatur.txt', '/content/KorpusARM1/stage02/hyNatur-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
print(len(LCorrected1))
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyNatur-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )

2134


In [16]:
LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyRecht.txt', '/content/KorpusARM1/stage02/hyRecht-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
print(len(LCorrected1))
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyRecht-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )

1534
